### ライブラリのimport

In [241]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import pandas as pd
%matplotlib inline
from preprocessing.initial_preprocessing import initial_preprocessing
from preprocessing.drop_emergency_suspicious_height_weight import drop_emergency_suspicious_height_weight
from preprocessing.remove_duplicate_ctdi import remove_duplicate_ctdi
from preprocessing.rename_and_drop_order import rename_and_drop_order_one_scan_ctdi
from preprocessing.remove_not_use_head_rest import remove_not_use_head_rest
from preprocessing.special_scan_exclude import special_scan_exclude

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 解析用の元データを読み込み

In [276]:
df = pd.read_excel('train_data.xlsx')

## データを前処理
* カラム名の変更
* 複数回スキャンについては,Mean CTDIvolが最大のみ残す
* 救急科などの怪しい身長と体重のデータを削除する

In [277]:
initial_preprocessing(df)
df = remove_duplicate_ctdi(df)

正常に処理が行われました。


In [243]:
initial_preprocessing(df)
df = remove_duplicate_ctdi(df)
df.reset_index(drop=True, inplace=True)
# df_allで後で、身長体重を代入する分を分けられるようにしておく
df_all = df
df = drop_emergency_suspicious_height_weight(df)

正常に処理が行われました。


### 身長・体重の最終確認: この部分は自分で確認しないと危険

In [244]:
index = df['height_cm'].sort_values(ascending=True)[:30].index.to_list()

In [245]:
df[['scanning length', 'height_cm', 'weight_kg', 'id', 'study_date', 'target region', 'Mean CTDIvol']].loc[index].head(2)

,scanning length,height_cm,weight_kg,id,study_date,target region,Mean CTDIvol
2220,852.06,101.0,39.8,75206,2021-04-21,Chest,6.45
994,796.77,101.0,39.8,75206,2021-05-18,Chest,7.38


In [246]:
# 最終的な怪しいデータに関しては直接データを確認して削除する
drop_ids = [75206, 9326611, 17049]
for ids in drop_ids:
    df = df[~(df['id'] == ids)]
df.reset_index(drop=True, inplace=True)

### 身長体重予測、線量予測、異常検知でも使用しない項目を削除する

In [247]:
unuse_cols = ['hospital_ward', 'modality', 'adult_child', 'department', 'preset_name']
df.drop(labels=unuse_cols, axis=1, inplace=True)

In [248]:
df.to_excel('preprocess_all_scan_data.xlsx')

In [249]:
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部CT'))]

* データに含まれている外れ値のデータはさらに削除しても良いかも

In [250]:
# 外れ値として取り扱うデータを追加
drop_accession_list = [1249030520220420, 1351460220230315, 1346485720230228, 1348835920230307,
                       1229782420220221]
for accession in drop_accession_list:
    df = df[~(df['accession'] == accession)]
df.reset_index(drop=True, inplace=True)

In [251]:
# 不適切な心臓関係のデータを取り除く
# 1.　管電圧が100kV, 2.肺静脈(カルト)で管電圧が140kVのもの,その他排除すべきデータ
#differenceが10以上のデータを確認した中で、以下のようなデータを削除 
# *脳でHead restを使用せずベッドで撮影したデータ 
# *オーダーに対して不適切な部位が撮影範囲に含まれている（脳CTで肩が含まれている等） 
# *腕を挙上していない（体幹部撮影の際） *上肢撮影で体幹が含まれている *金属等が装着されている *オーダーと撮影部位が異なる
unuse_accession = [1348557720230306, 1106913420210113, 1112271520210127, 1116921520210210, 1130557720210324,
                   1163646020210707, 1175490820210818, 1216316120220114, 1150306520210521, 1273513820220714, 
                   1301969120221017, 1283505320220817, 1315170020221125, 1184192220210917, 1293945820220920,
                   1257936320220523, 1190750720211013, 1219548120220124, 1364809020230425, 1117860420210215,
                   1249639320220422, 1335762120230126, 1291697020220912, 1359568020230411, 1190377920211012,
                   1262435120220608, 1347736320230303, 1289845120220906, 1223184520220202, 1330730720230113,
                   1327520120230104, 1197756320211109, 1129424720210322, 1309065120221107, 1343513920230218,
                   1198504120211111, 1208625320211216, 1358077220230406, 1130613820210324, 1223603420220203,
                   1197519820211108, 1105253820210108, 1140475720210420, 1335584020230126, 1293219420220916,
                   1144301520210430, 1111324820210125, 1338397620230204, 1164256820210709, 1323580920221220,
                   1339508720230207, 1229837920220221, 1193844120211025, 1201460220211121, 1124729420210308,
                   1111992520210127, 1323140120221219, 1290478220220908, 1319157220221207, 1218117220220119,
                   1276725620220725, 1122725320210302, 1206549320211209, 1208856820211216, 1214061320220106, 
                   1222147720220131, 1309403020221108, ]
for accession in unuse_accession:
    df = df[~(df['accession'] == accession)]
df.reset_index(drop=True, inplace=True)

### スキャンオーダーの名前を変更する

In [252]:
rename_and_drop_order_one_scan_ctdi(df)

### ヘッドレストを使っていない撮影を削除する

In [253]:
df_brain = pd.read_excel('brain_check_head_rest.xlsx')
df_inner = pd.read_excel('orbit_inner_ear.xlsx')
df = remove_not_use_head_rest(df, df_brain, df_inner)

### 特殊な四肢系の特殊なスキャンや本来用いない管電圧を取り除く

In [254]:
df = special_scan_exclude(df)

### 前処理が終了したデータを保存する。

In [255]:
df.to_excel('preprocessed_train_data.xlsx', index=False)